In [1]:
import numpy as np
import sys
import time
import h5py as h5
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

sys.path.append(r"C:\Users\haoyuan\Documents\GitHub\CrystalDiff")

from CrystalDiff import util, pulse, lclsutil, crystal

In [2]:
# ----------------------------------------------------------------------------------------------------------
#                       Step 1: Pulse
# ----------------------------------------------------------------------------------------------------------

energy_center = 10.
pre_length = 1e6

# Set up the pulse
my_pulse = pulse.GaussianPulse3D()
my_pulse.set_pulse_properties(central_energy=energy_center,
                              polar=[0., 1., 0.],
                              sigma_x=708.2581446128465,
                              sigma_y=708.2581446128465,
                              sigma_z=100.,
                              x0=np.array([0., 0., -pre_length]))

# ----------------------------------------------------------------------------------------------------------
#                       Step 2: Split
# ----------------------------------------------------------------------------------------------------------
dtheta = np.arctan(1.5 * 1e-3 / 5.)      # This is the deviation angle.

grating_list = [crystal.RectangleGrating(), crystal.RectangleGrating()]
grating_list[0].set_a(util.get_grating_period(dtheta=dtheta, klen_in=my_pulse.klen0) / 2)
grating_list[0].set_b(util.get_grating_period(dtheta=dtheta, klen_in=my_pulse.klen0) / 2)

grating_list[1].set_a(util.get_grating_period(dtheta=dtheta, klen_in=my_pulse.klen0) / 2)
grating_list[1].set_b(util.get_grating_period(dtheta=dtheta, klen_in=my_pulse.klen0) / 2)

# ----------------------------------------------------------------------------------------------------------
#                       Step 3: Delay Lines
# ----------------------------------------------------------------------------------------------------------
# Some meta data for the delay line.
reflect_num = 4
h_length = 2. * np.pi / (1.9201 * 1e-4)
hlen_vals = np.array([h_length, ] * reflect_num)

# Some crystal properties
chi0=complex(-0.97631E-05, 0.14871E-06)
chih_sigma=complex(0.59310E-05, -0.14320E-06)
chihbar_sigma=complex(0.59310E-05, -0.14320E-06)
chih_pi=complex(0.46945E-05, -0.11201E-06)
chihbar_pi=complex(0.46945E-05, -0.11201E-06)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal to fix the shearing
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_0 = 0.
bragg_angle_0 = np.radians(18.836) + 13e-6

branch_angle_0 = lclsutil.get_delay_line_angles(angle_offset=angle_offset_0,
                                                theta=bragg_angle_0 + np.pi / 2. ,
                                                rho=bragg_angle_0 - np.pi / 2. ,
                                                inclined_angle=np.radians(-10.))
surface_points_0 = np.zeros((reflect_num, 3), dtype=np.float64)

# Set the misalignment angle
misalign_branch_0_crystal_1 = [0., 0., 0.]
misalign_branch_0_crystal_2 = [0., 0., 0.]

# Initialize the crystals
shear_fix_crystal = lclsutil.get_crystal_list_delay_branch(hlen_vals=hlen_vals,
                                                        theta_vals=branch_angle_0[0],
                                                        rho_vals=branch_angle_0[1],
                                                        tau_vals=branch_angle_0[2],
                                                        surface_points=surface_points_0,
                                                        chi0=chi0,
                                                        chih_sigma=chih_sigma, chihbar_sigma=chihbar_sigma,
                                                        chih_pi=chih_pi, chihbar_pi=chihbar_pi)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal for branch  1
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_1 = dtheta
bragg_angle_1 = np.radians(18.836) + 13e-6

branch_angle_1 = lclsutil.get_delay_line_angles(angle_offset=angle_offset_1,
                                                theta=bragg_angle_1 + np.pi / 2. ,
                                                rho=bragg_angle_1 - np.pi / 2. ,
                                                inclined_angle=np.radians(10.))
surface_points_1 = np.zeros((reflect_num, 3), dtype=np.float64)

# Set the misalignment angle
misalign_branch_1_crystal_1 = [0., 0., 0.]
misalign_branch_1_crystal_2 = [0., 0., 0.]

# Initialize the crystals
crystal_list_1 = lclsutil.get_crystal_list_delay_branch(hlen_vals=hlen_vals,
                                                        theta_vals=branch_angle_1[0],
                                                        rho_vals=branch_angle_1[1],
                                                        tau_vals=branch_angle_1[2],
                                                        surface_points=surface_points_1,
                                                        chi0=chi0,
                                                        chih_sigma=chih_sigma, chihbar_sigma=chihbar_sigma,
                                                        chih_pi=chih_pi, chihbar_pi=chihbar_pi)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal for branch  2
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_2 = - dtheta
bragg_angle_2 = np.radians(18.836) + 13e-6

branch_angle_2 = lclsutil.get_delay_line_angles(angle_offset=angle_offset_2,
                                                theta=1.5 * np.pi - bragg_angle_2,
                                                rho=0.5 * np.pi - bragg_angle_2,
                                                inclined_angle=np.radians(10.))
surface_points_2 = np.zeros((reflect_num, 3), dtype=np.float64)

# Set the misalignment angle
misalign_branch_2_crystal_1 = [0., 0., 0.]
misalign_branch_2_crystal_2 = [0., 0., 0.]

# Initialize the crystals
crystal_list_2 = lclsutil.get_crystal_list_delay_branch(hlen_vals=hlen_vals,
                                                        theta_vals=branch_angle_2[0],
                                                        rho_vals=branch_angle_2[1],
                                                        tau_vals=branch_angle_2[2],
                                                        surface_points=surface_points_2,
                                                        chi0=chi0,
                                                        chih_sigma=chih_sigma, chihbar_sigma=chihbar_sigma,
                                                        chih_pi=chih_pi, chihbar_pi=chihbar_pi)


# ------------------------------------------------------
#   Define the positions
# ------------------------------------------------------
path_list_0 = [0., 1e5, 1e5, 1e5]
dist_shear_crystal_grating = 5e5

path_list_1 = [5e6 - 10e4, 1e5, 5e5, 1.05e5, 6e6, 1e6]
path_list_2 = [5e6, 1e5, 1e5, 1.05e5, 6e6, 1e6]
delay_time = 600.

In [3]:
for x in shear_fix_crystal:
    print(x.normal)

[ 0.         -0.94644239  0.32287273]
[-0.17364818  0.9320638  -0.31796757]
[-0.17364818  0.9320638   0.31796757]
[ 0.         -0.94644239 -0.32287273]


In [4]:
for x in crystal_list_1:
    print(x.normal)

[ 0.         -0.94634549  0.32315665]
[ 0.17364818  0.93196837 -0.31824718]
[0.17364818 0.93215915 0.31768794]
[ 0.         -0.94653921 -0.32258879]


In [5]:
# Get the positions for the shearing fix branch
(intersect_branch_0,
kout_branch_0) = lclsutil.get_point_with_definite_path(kin_vec = my_pulse.k0,
                                                       path_sections = path_list_0,
                                                       crystal_list = shear_fix_crystal)

# Adjust the path sections
(path_list_1,
 path_list_2) = lclsutil.get_split_delay_configuration(delay_time=delay_time,
                                                       fix_branch_path=path_list_2,
                                                       var_branch_path=path_list_1,
                                                       fix_branch_crystal=crystal_list_2,
                                                       var_branch_crystal=crystal_list_1,
                                                       grating_pair=grating_list,
                                                       kin=kout_branch_0[-1])

(intersect_branch_1,
 kout_brunch_1,
 intersect_branch_2,
 kout_brunch_2) = lclsutil.get_light_path(kin=kout_branch_0[-1],
                                          grating_list=grating_list,
                                          crystal_list_1=crystal_list_1,
                                          path_list_1=path_list_1,
                                          crystal_list_2=crystal_list_2,
                                          path_list_2=path_list_2)


In [6]:
kout_brunch_1

array([[ 5.01623523e-01, -3.63866795e-06,  5.06773076e+04],
       [ 5.01623523e-01,  1.52031886e+01,  5.06773076e+04],
       [ 5.01623523e-01,  3.09840214e+04,  4.01021215e+04],
       [ 2.50549742e-01,  1.52031832e+01,  5.06773076e+04],
       [-5.19319973e-04, -3.09599546e+04,  4.01207047e+04],
       [-5.19319973e-04,  1.52031850e+01,  5.06773076e+04],
       [-5.19319973e-04, -7.28419045e-06,  5.06773076e+04]])

In [7]:
kout_brunch_2

array([[ 5.01623523e-01, -3.63866795e-06,  5.06773076e+04],
       [ 5.01623523e-01, -1.52031959e+01,  5.06773076e+04],
       [ 5.01623523e-01, -3.09840214e+04,  4.01021215e+04],
       [ 2.50552171e-01, -1.52031905e+01,  5.06773076e+04],
       [-5.19320492e-04,  3.09599546e+04,  4.01207047e+04],
       [-5.19320492e-04, -1.52031923e+01,  5.06773076e+04],
       [-5.19320492e-04,  6.79206913e-09,  5.06773076e+04]])

In [8]:
def get_kout(kin, h, normal):
    kout = kin + h
    
    klen = np.sqrt(np.dot(kin, kin))
    
    gammah = np.dot(kin + h, normal) / klen
    alpha =  (2 * np.dot(kin, h) + np.dot(h, h)) / np.square(klen)

    if np.abs(-gammah - np.sqrt(gammah ** 2 - alpha) ) > np.abs(-gammah + np.sqrt(gammah ** 2 - alpha) ):
        momentum = klen *( -gammah + np.sqrt(gammah ** 2 - alpha))
    else:
        momentum = klen *( -gammah - np.sqrt(gammah ** 2 - alpha))
    
    kout += normal * momentum 
    return kout, klen ** 2 / np.dot(kout, kout)

In [12]:
kout = kout_branch_0[-1] + grating_list[0].base_wave_vector
print(kout)

miscuts = [np.array([-10e-6, -1e-5, 1e-5]),
           np.array([-10e-6, -1e-5, 1e-5]),
           np.array([-10e-6, -1e-5, 1e-5]),
           np.array([-10e-6, -1e-5, 1e-5])]


for x in range(4):
    
    rot_mat = Rotation.from_euler('xyz',miscuts[x], degrees=False)
    
    kout, ratio = get_kout(kin=kout,
                           h=rot_mat.as_dcm().dot(crystal_list_1[x].h),
                           normal= rot_mat.as_dcm().dot(crystal_list_1[x].normal))
    print(kout - grating_list[0].base_wave_vector, ratio)

[5.01623523e-01 1.52031886e+01 5.06773076e+04]
[2.97685378e-01 3.09696203e+04 4.01015019e+04] 1.0
[ 8.14026556e-02 -2.72456540e-05  5.06773076e+04] 1.0
[ 4.14787736e-01 -3.09743553e+04  4.01213239e+04] 1.0
[-5.15653047e-04 -2.54708570e-05  5.06773076e+04] 1.0


In [13]:
kout = kout_branch_0[-1] + grating_list[0].base_wave_vector
print(kout)

miscuts = [np.array([-10e-6, 1e-5, -1e-5]),
           np.array([-10e-6, 1e-5, -1e-5]),
           np.array([-10e-6, 1e-5, -1e-5]),
           np.array([-10e-6, 1e-5, -1e-5])]


for x in range(4):
    
    rot_mat = Rotation.from_euler('xyz',miscuts[x], degrees=False)
    
    kout, ratio = get_kout(kin=kout,
                           h=rot_mat.as_dcm().dot(crystal_list_1[x].h),
                           normal= rot_mat.as_dcm().dot(crystal_list_1[x].normal))
    print(kout - grating_list[0].base_wave_vector, ratio)

[5.01623523e-01 1.52031886e+01 5.06773076e+04]
[7.05561668e-01 3.09696203e+04 4.01015019e+04] 1.0
[8.14094513e-02 5.79859966e-06 5.06773076e+04] 0.9999999999999998
[-4.15823110e-01 -3.09743553e+04  4.01213239e+04] 1.0
[-5.19720019e-04  4.40692566e-06  5.06773076e+04] 1.0
